In [281]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import datetime

In [282]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from scipy.stats import zscore

In [283]:
# reading the raw file


Covid_df = pd.read_csv('Brazil_Covid.csv')



In [284]:
Covid_df.shape

(5644, 111)

In [341]:
null_series = (Covid_df.isnull().sum() != 0).index
target = 'SARS-Cov-2 exam result'
one_target = []

for col in null_series:
    i = Covid_df[Covid_df[col].notnull()][target].nunique()
    if i == 1:
        one_target.append(col)    

print(one_target)

['Fio2 (venous blood gas analysis)', 'Urine - Nitrite', 'Vitamin B12', 'Albumin']


In [342]:
for col in one_target:
    print(Covid_df[Covid_df[col].notnull()][target].unique())

[0]
[0]
[0]
[0]


In [285]:
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder() 


  
#Covid_df['SARS-Cov-2 exam result']= le.fit_transform(Covid_df['SARS-Cov-2 exam result']) 
Covid_df['SARS-Cov-2 exam result']= le.fit_transform(Covid_df['SARS-Cov-2 exam result']) 
#Covid_df['Urine - Color']= le.fit_transform(Covid_df['Urine - Color']) 

#Covid_df['Urine - Hyaline cylinders']= le.fit_transform(Covid_df['Urine - Hyaline cylinders']) 



#Covid_df['Respiratory Syncytial Virus']=le.fit_transform(Covid_df['Respiratory Syncytial Virus']) 

#Covid_df['Respiratory Syncytial Virus']= Covid_df['Respiratory Syncytial Virus'].str.replace("not_detected","0")
#Covid_df['Respiratory Syncytial Virus']= Covid_df['Respiratory Syncytial Virus'].str.replace("detected","1")



        #Covid_df[feature] = Covid_df[feature].str.replace("Urato Amorfo --+","2") # Replace object to numeric
        #Covid_df[feature] = Covid_df[feature].str.replace("Urato Amorfo +++","3")
       # Covid_df[feature] = Covid_df[feature].str.replace("Oxalato de Cálcio -++","4")       
       
        
        
        
        
        
        

In [286]:
for feature in Covid_df.columns: # Loop through all columns in the dataframe
    if Covid_df[feature].dtype == 'object': # Only apply for columns with categorical strings
        Covid_df[feature] = Covid_df[feature].str.replace("not_detected","0") # Replace object to numeric
        Covid_df[feature] = Covid_df[feature].str.replace("detected","1")
        Covid_df[feature] = Covid_df[feature].str.replace("negative","0") # Replace object to numeric
        Covid_df[feature] = Covid_df[feature].str.replace("positive","1")
        Covid_df[feature] = Covid_df[feature].str.replace("absent","0") # Replace object to numeric
        Covid_df[feature] = Covid_df[feature].str.replace("not_done","1")
        Covid_df[feature] = Covid_df[feature].str.replace("present","2")
        Covid_df[feature] = Covid_df[feature].str.replace("altered_coloring","4") # Replace object to numeric
        Covid_df[feature] = Covid_df[feature].str.replace("clear","3")
        Covid_df[feature] = Covid_df[feature].str.replace("lightly_cloudy","5") # Replace object to numeric
        Covid_df[feature] = Covid_df[feature].str.replace("cloudy","6")
        Covid_df[feature] = Covid_df[feature].str.replace("normal","7") 
        Covid_df[feature] = Covid_df[feature].str.replace("citrus_yellow","11")
        Covid_df[feature] = Covid_df[feature].str.replace("light_yellow","8") 
        Covid_df[feature] = Covid_df[feature].str.replace("yellow","9") 
        Covid_df[feature] = Covid_df[feature].str.replace("orange","10")
        Covid_df[feature] = Covid_df[feature].str.replace("Não Realizado","0") 
        Covid_df[feature] = Covid_df[feature].str.replace("<1000","999") 
        #Covid_df[feature] = Covid_df[feature].str.replace("Ausentes","1")  

In [287]:
Covid_df['SARS-Cov-2 exam result'].head(5000)

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
4970    0
4971    0
4972    0
4973    0
4974    1
4975    0
4976    0
4977    0
4978    1
4979    0
4980    1
4981    0
4982    0
4983    0
4984    0
4985    0
4986    0
4987    0
4988    0
4989    1
4990    0
4991    0
4992    0
4993    0
4994    0
4995    0
4996    0
4997    0
4998    0
4999    1
Name: SARS-Cov-2 exam result, Length: 5000, dtype: int32

In [288]:
Covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Columns: 111 entries, Patient ID to ctO2 (arterial blood gas analysis)
dtypes: float64(70), int32(1), int64(4), object(36)
memory usage: 4.8+ MB


In [289]:
Covid_df.isnull().sum()

Patient ID                                                  0
Patient age quantile                                        0
SARS-Cov-2 exam result                                      0
Patient addmited to regular ward (1=yes, 0=no)              0
Patient addmited to semi-intensive unit (1=yes, 0=no)       0
Patient addmited to intensive care unit (1=yes, 0=no)       0
Hematocrit                                               5041
Hemoglobin                                               5041
Platelets                                                5042
Mean platelet volume                                     5045
Red blood Cells                                          5042
Lymphocytes                                              5042
Mean corpuscular hemoglobin concentration (MCHC)         5042
Leukocytes                                               5042
Basophils                                                5042
Mean corpuscular hemoglobin (MCH)                        5042
Eosinoph

In [290]:
#print the count of the values
Covid_df['SARS-Cov-2 exam result'].value_counts()

0    5086
1     558
Name: SARS-Cov-2 exam result, dtype: int64

# Inference

1)we are seeing lot of feature which does not have any data there is no point is using those feture for our model and hence dropping those.

2)Hemoglobin play a vital part in blood and there are lot of records which do not have a preopr value. hence dropping those records

3) Albumin,Vitamin B12,Ferritin,Phosphor,Urine - Nitrite and Urine - Ketone Bodies has valid value for less significaton records and more null value and does not make sense to impute the null with mean or median



In [291]:
#creating a new datset with non misisng values
#CovidNew_df=Covid_df.dropna()

#Covid_df_NEW=Covid_df[Covid_df.columns[Covid_df.isnull().sum() = 25]]
# creating a new datset which removes all the columns which 


#Covid_df_new = Covid_df.dropna(axis=1)
Covid_df_new = Covid_df.dropna(how='all')
Covid_df_new = Covid_df_new.dropna(how='all',axis=1)

#df1 = df.dropna(how='all', axis=1)
#print(df1)

In [339]:
(Covid_df_new.isnull().sum() / Covid_df_new.shape[0]).sort_values(ascending=False)

Fio2 (venous blood gas analysis)                         0.998342
Arteiral Fio2                                            0.966833
ctO2 (arterial blood gas analysis)                       0.955224
pO2 (arterial blood gas analysis)                        0.955224
HCO3 (arterial blood gas analysis)                       0.955224
Total CO2 (arterial blood gas analysis)                  0.955224
pH (arterial blood gas analysis)                         0.955224
Base excess (arterial blood gas analysis)                0.955224
pCO2 (arterial blood gas analysis)                       0.955224
Hb saturation (arterial blood gases)                     0.955224
Arterial Lactic Acid                                     0.955224
Magnesium                                                0.933665
Urine - Protein                                          0.923715
Urine - Esterase                                         0.920398
Ionized calcium                                          0.917081
Urine - Hy

In [340]:
null_series = (Covid_df_new.isnull().sum() != 0).index
target = 'SARS-Cov-2 exam result'
one_target = []

for col in null_series:
    i = Covid_df_new[Covid_df_new[col].notnull()][target].nunique()
    if i == 1:
        one_target.append(col)    

print(one_target)

['Fio2 (venous blood gas analysis)']


In [292]:
Covid_df_new = Covid_df_new.dropna(axis=0, subset=['Hemoglobin'])

In [293]:
Covid_df_new = Covid_df_new.drop(["Albumin","Ferritin","Lipase dosage","Vitamin B12","Urine - Crystals","Phosphor","Urine - Nitrite","Urine - Ketone Bodies"],axis = 1)

In [294]:
#print the count of the values
Covid_df_new.shape

(603, 98)

In [295]:
Covid_df_new.isnull().sum()

Patient ID                                                 0
Patient age quantile                                       0
SARS-Cov-2 exam result                                     0
Patient addmited to regular ward (1=yes, 0=no)             0
Patient addmited to semi-intensive unit (1=yes, 0=no)      0
Patient addmited to intensive care unit (1=yes, 0=no)      0
Hematocrit                                                 0
Hemoglobin                                                 0
Platelets                                                  1
Mean platelet volume                                       4
Red blood Cells                                            1
Lymphocytes                                                1
Mean corpuscular hemoglobin concentration (MCHC)           1
Leukocytes                                                 1
Basophils                                                  1
Mean corpuscular hemoglobin (MCH)                          1
Eosinophils             

# Handling null value

There are Total of 558 Postive case and 5086 negative case . we have many miisng values and to handle it let create datset for positive and negative seperatly and handle it. As Hemoglobin forms a a vital stats we cant not input it with mean and median and hence ignoring those records.

In [296]:
Covid_df_new=Covid_df_new.drop(["Patient ID"],axis = 1)

In [297]:
#X = main_df.drop(['greatstone_rating','fund_id'],axis = 1)
#y = main_df['greatstone_rating']

In [298]:
Covid_Postive_df=Covid_df_new[Covid_df_new['SARS-Cov-2 exam result'] == 1]

In [299]:
Covid_Postive_df.shape

(83, 97)

In [300]:
Covid_Postive_df.isnull().sum()

Patient age quantile                                      0
SARS-Cov-2 exam result                                    0
Patient addmited to regular ward (1=yes, 0=no)            0
Patient addmited to semi-intensive unit (1=yes, 0=no)     0
Patient addmited to intensive care unit (1=yes, 0=no)     0
Hematocrit                                                0
Hemoglobin                                                0
Platelets                                                 0
Mean platelet volume                                      2
Red blood Cells                                           0
Lymphocytes                                               0
Mean corpuscular hemoglobin concentration (MCHC)          0
Leukocytes                                                0
Basophils                                                 0
Mean corpuscular hemoglobin (MCH)                         0
Eosinophils                                               0
Mean corpuscular volume (MCV)           

In [301]:
Covid_Postive_Regular_df=Covid_Postive_df[Covid_Postive_df['Patient addmited to regular ward (1=yes, 0=no)'] == 1]

In [302]:
Covid_Postive_Regular_df.shape

(27, 97)

In [303]:
Covid_Postive_semi_intensive_df=Covid_Postive_df[Covid_Postive_df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1]

In [304]:
Covid_Postive_semi_intensive_df.shape

(8, 97)

In [305]:
Covid_Postive_intensive_df=Covid_Postive_df[Covid_Postive_df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1]

In [306]:
Covid_Postive_intensive_df.shape

(8, 97)

In [307]:
Covid_Postive_Other_df=Covid_Postive_df[(Covid_Postive_df['Patient addmited to regular ward (1=yes, 0=no)'] == 0) &
(Covid_Postive_df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 0) &
(Covid_Postive_df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 0)
]



In [308]:
Covid_Postive_Other_df.shape

(40, 97)

In [309]:
def groupMeanValue(group):
    group = group.fillna(group.median())
    return group



In [310]:
Covid_Postive_Other_df = Covid_Postive_Other_df.transform(groupMeanValue)


In [311]:
Covid_Postive_Other_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [312]:

Covid_Postive_Regular_df = Covid_Postive_Regular_df.transform(groupMeanValue)
Covid_Postive_Regular_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [313]:
Covid_Postive_semi_intensive_df = Covid_Postive_semi_intensive_df.transform(groupMeanValue)
Covid_Postive_semi_intensive_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [314]:
Covid_Postive_intensive_df = Covid_Postive_intensive_df.transform(groupMeanValue)
Covid_Postive_intensive_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [315]:

Covid_Postive_New_df = Covid_Postive_intensive_df.append([Covid_Postive_semi_intensive_df, Covid_Postive_Regular_df,Covid_Postive_Other_df])

In [316]:
Covid_Postive_New_df.shape

(83, 97)

In [317]:
Covid_Postive_New_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [318]:
Covid_Negative_df=Covid_df_new[Covid_df_new['SARS-Cov-2 exam result'] == 0]

In [319]:
Covid_Negative_df.shape

(520, 97)

In [320]:
Covid_Negative_df.isnull().sum()

Patient age quantile                                       0
SARS-Cov-2 exam result                                     0
Patient addmited to regular ward (1=yes, 0=no)             0
Patient addmited to semi-intensive unit (1=yes, 0=no)      0
Patient addmited to intensive care unit (1=yes, 0=no)      0
Hematocrit                                                 0
Hemoglobin                                                 0
Platelets                                                  1
Mean platelet volume                                       2
Red blood Cells                                            1
Lymphocytes                                                1
Mean corpuscular hemoglobin concentration (MCHC)           1
Leukocytes                                                 1
Basophils                                                  1
Mean corpuscular hemoglobin (MCH)                          1
Eosinophils                                                1
Mean corpuscular volume 

In [321]:
Covid_Negative_Regular_df=Covid_Negative_df[Covid_Negative_df['Patient addmited to regular ward (1=yes, 0=no)'] == 0]

In [322]:
Covid_Negative_Regular_df.isnull().sum()

Patient age quantile                                       0
SARS-Cov-2 exam result                                     0
Patient addmited to regular ward (1=yes, 0=no)             0
Patient addmited to semi-intensive unit (1=yes, 0=no)      0
Patient addmited to intensive care unit (1=yes, 0=no)      0
Hematocrit                                                 0
Hemoglobin                                                 0
Platelets                                                  1
Mean platelet volume                                       2
Red blood Cells                                            1
Lymphocytes                                                1
Mean corpuscular hemoglobin concentration (MCHC)           1
Leukocytes                                                 1
Basophils                                                  1
Mean corpuscular hemoglobin (MCH)                          1
Eosinophils                                                1
Mean corpuscular volume 

In [323]:
Covid_Negative_semi_intensive_df=Covid_Negative_df[Covid_Negative_df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 0]

In [324]:
Covid_Negative_semi_intensive_df.isnull().sum()

Patient age quantile                                       0
SARS-Cov-2 exam result                                     0
Patient addmited to regular ward (1=yes, 0=no)             0
Patient addmited to semi-intensive unit (1=yes, 0=no)      0
Patient addmited to intensive care unit (1=yes, 0=no)      0
Hematocrit                                                 0
Hemoglobin                                                 0
Platelets                                                  1
Mean platelet volume                                       2
Red blood Cells                                            1
Lymphocytes                                                1
Mean corpuscular hemoglobin concentration (MCHC)           1
Leukocytes                                                 1
Basophils                                                  1
Mean corpuscular hemoglobin (MCH)                          1
Eosinophils                                                1
Mean corpuscular volume 

In [325]:
Covid_Negative_intensive_df=Covid_Negative_df[Covid_Negative_df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 0]

In [326]:
Covid_Negative_intensive_df.isnull().sum()

Patient age quantile                                       0
SARS-Cov-2 exam result                                     0
Patient addmited to regular ward (1=yes, 0=no)             0
Patient addmited to semi-intensive unit (1=yes, 0=no)      0
Patient addmited to intensive care unit (1=yes, 0=no)      0
Hematocrit                                                 0
Hemoglobin                                                 0
Platelets                                                  1
Mean platelet volume                                       2
Red blood Cells                                            1
Lymphocytes                                                1
Mean corpuscular hemoglobin concentration (MCHC)           1
Leukocytes                                                 1
Basophils                                                  1
Mean corpuscular hemoglobin (MCH)                          1
Eosinophils                                                1
Mean corpuscular volume 

In [327]:
Covid_Negative_Other_df=Covid_Negative_df[(Covid_Negative_df['Patient addmited to regular ward (1=yes, 0=no)'] == 0) &
(Covid_Negative_df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 0) &
(Covid_Negative_df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 0)
]

In [328]:
Covid_Negative_Other_df = Covid_Postive_Other_df.transform(groupMeanValue)
Covid_Negative_Other_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [329]:
Covid_Negative_Regular_df = Covid_Postive_Regular_df.transform(groupMeanValue)
Covid_Negative_Regular_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [330]:
Covid_Negative_intensive_df = Covid_Negative_intensive_df.transform(groupMeanValue)
Covid_Negative_intensive_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [331]:
Covid_Negative_semi_intensive_df = Covid_Negative_semi_intensive_df.transform(groupMeanValue)
Covid_Negative_semi_intensive_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [332]:
Covid_Negative_New_df = Covid_Negative_intensive_df.append([Covid_Negative_semi_intensive_df, Covid_Negative_Regular_df,Covid_Negative_Other_df])

In [333]:
Covid_Negative_New_df.shape

(1052, 97)

In [335]:
# combining both positive and negative valida datset

Covid_Input_df = Covid_Negative_New_df.append([Covid_Postive_New_df])




In [336]:
Covid_Input_df.shape

(1135, 97)

In [337]:
Covid_Input_df.isna().sum()

Patient age quantile                                     0
SARS-Cov-2 exam result                                   0
Patient addmited to regular ward (1=yes, 0=no)           0
Patient addmited to semi-intensive unit (1=yes, 0=no)    0
Patient addmited to intensive care unit (1=yes, 0=no)    0
Hematocrit                                               0
Hemoglobin                                               0
Platelets                                                0
Mean platelet volume                                     0
Red blood Cells                                          0
Lymphocytes                                              0
Mean corpuscular hemoglobin concentration (MCHC)         0
Leukocytes                                               0
Basophils                                                0
Mean corpuscular hemoglobin (MCH)                        0
Eosinophils                                              0
Mean corpuscular volume (MCV)                           

In [338]:
sns.pairplot(Covid_Input_df, diag_kind='kde',hue = 'status') # plotting pairplot to understand bi / multilinearlity if any.

KeyError: 'status'

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000001CF3D0E6AE8> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000001CF3D174950> (for post_execute):


KeyboardInterrupt: 

In [ ]:
#Plotting the correlation
Covid_Input_df.corr()

In [ ]:
import seaborn as sns
%matplotlib inline
from matplotlib import rcParams

from matplotlib import pyplot as plt
plt.figure(figsize=(25, 25))
ax = sns.heatmap(Covid_Input_df.corr(), vmax=.8, square=True, fmt='.2f', annot=True, linecolor='white', linewidths=0.01)
plt.title('Correlation')
plt.show()

In [ ]:
X = Covid_df_new.drop(["SARS-Cov-2 exam result","Patient ID"],axis = 1)
Y = Covid_df_new["SARS-Cov-2 exam result"]

Feature selection to improve accuracy

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,Y)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.
plt.show()

# Inference

There are Total of 558 Postive case and 5086 negative case . we have many miisng values and to handle it let create datset for positive and negative seperatly and handle it 

In [ ]:
Covid_Postive_df=Covid_df[Covid_df['SARS-Cov-2 exam result'] == 1]

In [ ]:
Covid_Postive_df.shape

In [ ]:
Covid_Postive_df.isnull().sum()

In [ ]:
Covid_Postive_df.shape

In [ ]:
Covid_Neative_df=Covid_df[Covid_df['SARS-Cov-2 exam result'] == 0]

In [ ]:
Covid_Neative_df.shape